In [1]:
#Connexion à la bdd
from py2neo import Graph, authenticate, Relationship
graph = Graph()
authenticate("localhost:7474","neo4j", "admin")

## Quel comics contiennent le plus de personnages?


In [13]:
graph.cypher.execute('''
MATCH (c)-[r]->(x)
WITH DISTINCT(x.name) AS COMICS, COUNT(r) AS NB_APP
ORDER BY NB_APP DESC 
RETURN COMICS, NB_APP LIMIT 10
''')

    | COMICS               | NB_APP
----+----------------------+--------
  1 | COC 1                |    111
  2 | MARVEL MYSTERY COMIC |     92
  3 | IW 3                 |     91
  4 | IW 1                 |     90
  5 | H2 279               |     87
  6 | IW 4                 |     80
  7 | IW 2                 |     76
  8 | MAXSEC 3             |     72
  9 | FF 370               |     63
 10 | IW 6                 |     60

## Quel personnage apparait le plus souvant ?

In [14]:
graph.cypher.execute('''
MATCH (c)-[r]->()
WITH DISTINCT(c.name) AS PERSONNAGE, COUNT(r) AS NB_APP
ORDER BY NB_APP DESC
RETURN PERSONNAGE, NB_APP LIMIT 10
''')

    | PERSONNAGE           | NB_APP
----+----------------------+--------
  1 | SPIDER-MAN/PETER PAR |   1625
  2 | CAPTAIN AMERICA      |   1367
  3 | IRON MAN/TONY STARK, |   1168
  4 | THING/BENJAMIN J. GR |    990
  5 | THOR/DR. DONALD BLAK |    965
  6 | HUMAN TORCH/JOHNNY S |    908
  7 | MR. FANTASTIC/REED R |    875
  8 | HULK/DR. ROBERT BRUC |    841
  9 | WOLVERINE/LOGAN,     |    820
 10 | INVISIBLE WOMAN/SUE, |    782

## Quel personnage a le plus d'interactions avec les autres?

In [20]:
graph.cypher.execute('''
MATCH (c1)-->(n)<--(c2)
WITH COUNT(DISTINCT(c2.name)) AS NB_RELATIONS, c1.name AS PERSONNAGE
ORDER BY NB_RELATIONS DESC
RETURN PERSONNAGE, NB_RELATIONS LIMIT 10
''')

    | PERSONNAGE           | NB_RELATIONS
----+----------------------+--------------
  1 | CAPTAIN AMERICA      |         1906
  2 | SPIDER-MAN/PETER PAR |         1737
  3 | IRON MAN/TONY STARK, |         1522
  4 | THING/BENJAMIN J. GR |         1416
  5 | MR. FANTASTIC/REED R |         1379
  6 | WOLVERINE/LOGAN,     |         1371
  7 | HUMAN TORCH/JOHNNY S |         1361
  8 | SCARLET WITCH/WANDA, |         1325
  9 | THOR/DR. DONALD BLAK |         1289
 10 | BEAST/HENRY &HANK& P |         1267

In [26]:
#Exemple: CAP AMERICA
graph.cypher.execute('''
MATCH (c1)-->()<--(c2)
WHERE c1.name = "CAPTAIN AMERICA"
WITH DISTINCT(c2.name) AS PERSONNAGE, COUNT(c2.name) AS NB
ORDER BY NB DESC
RETURN PERSONNAGE, NB
''')

      | PERSONNAGE           | NB 
------+----------------------+-----
    1 | IRON MAN/TONY STARK, | 446
    2 | THOR/DR. DONALD BLAK | 386
    3 | VISION,              | 385
    4 | WASP/JANET VAN DYNE, | 384
    5 | SCARLET WITCH/WANDA, | 374
    6 | HAWK                 | 322
    7 | ANT-MAN/DR. HENRY J. | 295
    8 | JARVIS, EDWIN,       | 244
    9 | WONDER MAN/SIMON WIL | 216
   10 | FALCON/SAM WILSON    | 189
   11 | HERCULES [GREEK GOD] | 182
   12 | SHE-HULK/JENNIFER WA | 172
   13 | BEAST/HENRY &HANK& P | 169
   14 | THING/BENJAMIN J. GR | 168
   15 | MR. FANTASTIC/REED R | 166
   16 | QUICKSILVER/PIETRO M | 165
   17 | SUB-MARINER/NAMOR MA | 158
   18 | HUMAN TORCH/JOHNNY S | 158
   19 | FURY, COL. NICHOLAS, | 155
   20 | INVISIBLE WOMAN/SUE, | 149
   21 | SPIDER-MAN/PETER PAR | 145
   22 | BLACK WIDOW/NATASHA, | 143
   23 | BLACK PANTHER/T'CHAL | 131
   24 | CARTER, SHARON       | 126
   25 | BLACK KNIGHT V/DANE, | 119
   26 | CAPTAIN MARVEL II/MO | 119
   27 | HULK/DR. RO

## Quels personnages apparaissent le plus souvant ensembles?

In [30]:
graph.cypher.execute('''
    MATCH (m:character)-[r:APPEARS_IN]->(:comics)
    WITH m, COUNT(DISTINCT r) AS nb_tar
    SET m.NB_COMICS = nb_tar;
''')


In [34]:
from py2neo.packages.httpstream import http
http.socket_timeout = 9999   # fix for the socket timeout error

#calul des distances de Jaccard
graph.cypher.execute('''
    MATCH (c1:character)-[r1:APPEARS_IN]->(n:comics)<-[r2:APPEARS_IN]-(c2:character)
    WITH c1, c2,tofloat(c1.NB_COMICS) AS c1_tar,tofloat(c2.NB_COMICS) AS c2_tar, tofloat(COUNT(DISTINCT n)) AS nb_tar
    WHERE c1.name <> c2.name
    WITH c1,c2,(1-nb_tar/(c1_tar + c2_tar - nb_tar)) AS Jaccard
    CREATE UNIQUE (c1)-[:DISTANCE {JACCARD:Jaccard}]-(c2);
''')

In [41]:
graph.cypher.execute('''
    MATCH (c1:character)-[r:DISTANCE]->(c2:character)
    WITH c2.name AS PERSONNAGE, r.JACCARD AS COOCURENCE
    WHERE c1.name = "CAPTAIN AMERICA"
    WITH PERSONNAGE, COOCURENCE
    ORDER BY COOCURENCE 
    RETURN PERSONNAGE, COOCURENCE LIMIT 10;
''')

    | PERSONNAGE           | COOCURENCE    
----+----------------------+----------------
  1 | WASP/JANET VAN DYNE, | 0.756035578145
  2 | SCARLET WITCH/WANDA, | 0.771533292608
  3 | IRON MAN/TONY STARK, | 0.786500718047
  4 | THOR/DR. DONALD BLAK | 0.801644398767
  5 | ANT-MAN/DR. HENRY J. | 0.820231566118
  6 | JARVIS, EDWIN,       | 0.839684625493
  7 | WONDER MAN/SIMON WIL | 0.860284605433
  8 | FALCON/SAM WILSON    | 0.864903502502
  9 | FURY, COL. NICHOLAS, | 0.907957244656
 10 | SUB-MARINER/NAMOR MA | 0.909403669725

In [45]:
#extraction des paires et leur distance
pers_paires = graph.cypher.execute('''
    MATCH (c1:character)-[d:DISTANCE]->(c2:character)
    WITH c1,c2,d
    WHERE c1.name <> c2.name
    RETURN c1.name AS PERSONNAGE1, c2.name AS PERSONNAGE2, d.JACCARD AS DISTANCE;
''')

In [46]:
pers_paires.one

 PERSONNAGE1          | PERSONNAGE2 | DISTANCE      
----------------------+-------------+----------------
 24-HOUR MAN/EMMANUEL | OLD SKULL   | 0.947368421053

In [55]:
#Creation de la liste de reference des personnages
pers = []
for pair in pers_paires:
    if pair[0] not in pers:
        pers.append(pair[0])
    if pair[1] not in pers:
        pers.append(pair[1])
pers[:5]

[u'24-HOUR MAN/EMMANUEL',
 u'OLD SKULL',
 u"G'RATH",
 u"M'SHULLA",
 u'3-D MAN/CHARLES CHAN']

In [60]:
#chaque personnage est nommé une seule fois?
len(pers) == len(set(pers))


True

In [62]:
#on stock les personnages dans une dataframe
import pandas as pd
pers_df = pd.DataFrame(pers, columns=["Personnage"])

In [63]:
#creation d'une matrice
import numpy as np
len_mat = len(pers)
mat = np.empty([len_mat,len_mat], dtype=float)
mat.fill(-100.0)

In [65]:
#remplissage de la matrice en suivant l'ordre des personnages de la df
for pair in pers_paires:
    mat[np.where(pers_df.Personnage == pair[0]), np.where(pers_df.Personnage == pair[1])] = -100 * pair[2]


In [66]:
#la distance d'un personnage avec lui-meme est 0
np.fill_diagonal(mat,-0.0)

In [67]:
mat[:4,:4]

array([[  -0.        ,  -94.73684211,   -0.        ,  -95.        ],
       [-100.        ,   -0.        ,  -94.73684211, -100.        ],
       [-100.        , -100.        ,   -0.        , -100.        ],
       [-100.        ,   -5.        ,  -95.        ,   -0.        ]])

In [70]:
np.median(mat)

-100.0

In [72]:
#Clustering par affinity propagation 

from sklearn.cluster import AffinityPropagation
from sklearn import metrics

af = AffinityPropagation(preference = np.median(mat), affinity = "precomputed")
af.fit(mat)

AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=-100.0, verbose=False)

In [75]:
# evaluation par coefficient silouhette 
pers_df["cluster"] = af.labels_
score = metrics.silhouette_score(mat , af.labels_, metric="euclidean")
score

0.078182795402719274

Un score proche de 0 indique que les cluster se chevauchent. Il n'y a pas une bonne séparation en utilisant l'affinity propagation.

In [78]:
#Kmeans clustering
from sklearn.cluster import KMeans
km_scores=[]
clusters = range(2,150)
for cl in clusters:
    km_model =  KMeans(n_clusters=cl, n_jobs=-1).fit(mat)
    score = metrics.silhouette_score(mat,km_model.labels_, metric="euclidean")
    km_scores.append([cl,score])
    

In [79]:
km_scores

[[2, 0.46513490116132411],
 [3, -0.0022964389179086762],
 [4, -0.010502840641978654],
 [5, -0.029811305072783931],
 [6, 0.082927834471675982],
 [7, 0.014044246567951103],
 [8, -0.046768328401337558],
 [9, 0.13052833524592497],
 [10, 0.0066209620994213724],
 [11, 0.0002206648026330776],
 [12, -0.090532268884195341],
 [13, -0.025380050357485352],
 [14, 0.051190567840094604],
 [15, -0.044909426357830205],
 [16, -0.015889986034226783],
 [17, -0.061745459008327903],
 [18, 0.05050256885062266],
 [19, -0.06108325962215766],
 [20, -0.012289156125869527],
 [21, 0.076787167173787862],
 [22, -0.014109350486518501],
 [23, 0.01468587278172178],
 [24, -0.012033709531005804],
 [25, 0.060044080420561315],
 [26, -0.026841723204640275],
 [27, 0.066785680940391695],
 [28, -0.022980093601683859],
 [29, 0.034697378152453717],
 [30, 0.015775347607795594],
 [31, 0.0059132459544986695],
 [32, -0.028825687995744372],
 [33, -0.052139531762222621],
 [34, 0.077416145759170996],
 [35, 0.0095607041500757224],
 [36,